# Import

In [114]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [115]:
DAYS_0 = pd.Timedelta(days=0)
DAYS_1 = pd.Timedelta(days=1)
DAYS_14 = pd.Timedelta(days=14)

In [116]:
FILE_NAME = "PQ2MON - Orders - Weeks -1 to -109 (1).xls.xlsx"
DF = pd.read_excel(FILE_NAME)
DF.head(5)

,Client Grouping1,Customer Group,Requested Mode,Priority,Requested Trailer Class,Shipper Region3,Consignee Region3,Lane ID - City to City,Start Date,Completion Date,Order #,Avg. Weekly Frequency
0,Corporate,3M Canada Company,ROAD,Standard,DRY,ON2TOR,PQ2S,"MILTON,ON/ to DRUMMONDVILLE,PQ/",2019-04-16,2019-04-18,3790088,0
1,Corporate,3M Canada Company,ROAD,Standard,DRY,ON2TOR,PQ2S,"MILTON,ON/ to DRUMMONDVILLE,PQ/",2019-05-07,2019-05-09,3810953,0
2,Corporate,3M Canada Company,ROAD,Standard,DRY,ON2TOR,PQ2S,"MILTON,ON/ to DRUMMONDVILLE,PQ/",2019-06-07,2019-06-10,3841144,0
3,Corporate,3M Canada Company,ROAD,Standard,DRY,ON2TOR,PQ2S,"MILTON,ON/ to DRUMMONDVILLE,PQ/",2019-07-09,2019-07-11,3869973,0
4,Corporate,3M Canada Company,ROAD,Standard,DRY,PQ2MON,USMWIA,"STE THERESE,PQ/TE to PRAIRIE DU CHIEN,WI/",2020-02-26,2020-03-02,4066936,0


# Helper Functions

## Clean Data

In [117]:
def get_df_clean(df,drop=["Customer Group","Requested Mode","Order #","Avg. Weekly Frequency","Lane ID - City to City","Client Grouping1"]):
    df_PQ = df[df['Consignee Region3'].str.contains('PQ') | df['Shipper Region3'].str.contains('PQ')]
    df_clean = df_PQ.drop(columns=drop)
    return df_clean    

In [118]:
# Demo
DF_CLEAN = get_df_clean(DF)
DF_CLEAN

,Priority,Requested Trailer Class,Shipper Region3,Consignee Region3,Start Date,Completion Date
0,Standard,DRY,ON2TOR,PQ2S,2019-04-16,2019-04-18
1,Standard,DRY,ON2TOR,PQ2S,2019-05-07,2019-05-09
2,Standard,DRY,ON2TOR,PQ2S,2019-06-07,2019-06-10
3,Standard,DRY,ON2TOR,PQ2S,2019-07-09,2019-07-11
4,Standard,DRY,PQ2MON,USMWIA,2020-02-26,2020-03-02
...,...,...,...,...,...,...
23864,Standard,DRY,PQ2MON,BC2VAN,2020-06-18,2020-06-22
23865,Standard,REEFER,PQ2MON,ABNEDM,2020-02-12,2020-02-18
23866,Standard,REEFER,PQ2MON,ABNEDM,2020-02-12,2020-02-20
23867,Standard,REEFER,PQ2MON,ABNEDM,2020-02-13,2020-02-18


## Getting the statistics

In [119]:
# Get the counts for a schedule, such as inbound and out bound figures. Also calculates the imbalance levels
def get_df_count(df,start_date=None,end_date=None,fill_missing=False):    
    if start_date == None: start_date = df.loc[:,"Start Date":"Completion Date"].min().min() # This is not a mistake
    if end_date == None: end_date = df.loc[:,"Start Date":"Completion Date"].max().max() # This is not a mistake
    df_out = df[df['Shipper Region3'].str.contains('PQ')]['Start Date'].value_counts().sort_index(axis=0)
    df_in = df[df['Consignee Region3'].str.contains('PQ')]['Completion Date'].value_counts().sort_index(axis=0)
    df_count = pd.concat([df_out, df_in],axis=1).fillna(0, downcast='infer').rename(columns={"Start Date": "Outbound", "Completion Date": "Inbound"})
    if fill_missing:
        all_days  = [start_date + i*DAYS_1 for i in range((end_date-start_date).days+1)]
        df_count = pd.DataFrame(index=dates).join(df_count,how='outer').fillna(0)    
    df_count["Imbalance"] = df_count["Inbound"] - df_count["Outbound"]
    df_count["cImbalance"] = df_count["Imbalance"].cumsum()
    return df_count[start_date:end_date]

# Get the imbalance level of the date
def get_imb(df,date,col="Imbalance"):
    if col == "Imbalance" or col == "cImbalance":
        return df.loc[date,col]

In [120]:
#Demo
DF_COUNT = get_df_count(DF_CLEAN)
DF_COUNT

,Outbound,Inbound,Imbalance,cImbalance
2019-03-13,1,0,-1,-1
2019-03-19,2,0,-2,-3
2019-03-20,2,0,-2,-5
2019-03-21,1,0,-1,-6
2019-03-22,12,0,-12,-18
...,...,...,...,...
2021-04-20,21,20,-1,253
2021-04-21,12,25,13,266
2021-04-22,10,30,20,286
2021-04-23,11,24,13,299


In [121]:
#Demo
start_date = pd.Timestamp(2019,3,13,0)
end_date = pd.Timestamp(2019,3,19,0)

# df_count = get_df_count(DF_CLEAN,start_date,end_date)
df_count = get_df_count(DF_CLEAN,start_date,end_date,fill_missing=True)
df_count

,Outbound,Inbound,Imbalance,cImbalance
2019-03-13,1.0,0.0,-1.0,-1.0
2019-03-14,0.0,0.0,0.0,-1.0
2019-03-15,0.0,0.0,0.0,-1.0
2019-03-16,0.0,0.0,0.0,-1.0
2019-03-17,0.0,0.0,0.0,-1.0
2019-03-18,0.0,0.0,0.0,-1.0
2019-03-19,2.0,0.0,-2.0,-3.0


In [123]:
# Demo
df = DF_COUNT
date1 = pd.Timestamp(2020,1,1,0)
date2 = pd.Timestamp(2020,1,7,0)

get_imb(df,date1)
# get_imb(df,date2)
# get_imb(df,date2,col="cImbalance")

-3

In [124]:
# Get all the orders that start and/or completes on a specific day
# 'outer' means or, 'inner' means and
def get_df_day(df,date,col=['Start Date','Completion Date'],how='outer'): 
    cond = False
    if type(col) != list:
        cond = df[col] == date
    elif len(col) == 1:
        cond = df[col[0]] == date
    elif len(col) == 2:
        if how == 'outer':
            cond = (df[col[0]] == date) | (df[col[1]] == date)
        elif how == 'inner':
            cond = (df[col[0]] == date) & (df[col[1]] == date)
    return df[cond].sort_values(col)

# Get all orders that starts and/or completes in a specific time span
def get_df_span(df,start_date=None,end_date=None,how='outer'):
    if start_date == None: start_date = df.loc[:,"Start Date":"Completion Date"].min().min() # This is not a mistake
    if end_date == None: end_date = df.loc[:,"Start Date":"Completion Date"].max().max() # This is not a mistake
    cond1 = (df["Start Date"] >= start_date) & (df["Start Date"] <= end_date)
    cond2 = (df["Completion Date"] >= start_date) & (df["Completion Date"] <= end_date)
    if how == 'outer': 
        return df[cond1 | cond2]
    elif how == 'inner':
        return df[cond1 & cond2]

In [125]:
# Demo
df = DF_CLEAN
date1 = pd.Timestamp(2020,1,1,0)
date2 = pd.Timestamp(2020,1,7,0)

get_df_day(df,date1)
# get_df_day(df,date1,'Start Date')
# get_df_day(df,date2,'Completion Date')
# get_df_day(df,date2,['Start Date', 'Completion Date'],'inner')

,Priority,Requested Trailer Class,Shipper Region3,Consignee Region3,Start Date,Completion Date
20765,Standard,REEFER,PQ2MON,ON2TOR,2020-01-01,2020-01-01
17496,Expedited,DRY,PQ2MON,MB2WIN,2020-01-01,2020-01-02
20707,Standard,REEFER,PQ2MON,ON2N,2020-01-01,2020-01-02
9097,Standard,DRY,USEASE,PQ2MON,2020-01-01,2020-01-03


In [126]:
# Demo
df = DF_CLEAN
date1 = pd.Timestamp(2020,1,1,0)
date2 = pd.Timestamp(2020,1,7,0)

get_df_span(df,date1,date2,'outer')
# get_df_span(df,start_date=None,end_date=date2,how='inner')
# get_df_span(df,date1,date2,'inner')

,Priority,Requested Trailer Class,Shipper Region3,Consignee Region3,Start Date,Completion Date
15,Standard,DRY,PQ2OTT,PQ2MON,2019-12-30,2020-01-03
58,Standard,DRY,USMNMI,PQ2S,2020-01-03,2020-01-07
619,Standard,DRY,PQ2MON,USMEON,2020-01-06,2020-01-08
640,Standard,DRY,PQ2MON,USMEON,2020-01-03,2020-01-06
650,Standard,DRY,PQ2MON,USMEON,2020-01-06,2020-01-08
...,...,...,...,...,...,...
22917,Expedited,HEATER,PQ2MON,MB2WIN,2020-01-03,2020-01-05
23268,Standard,DRY,PQ2MON,USSCTN,2019-12-23,2020-01-06
23269,Standard,DRY,PQ2MON,USSCTN,2019-12-27,2020-01-02
23270,Standard,DRY,PQ2MON,USSCTN,2020-01-03,2020-01-08


For get_cand_orders this is a function I wrote first so it might be a lil confusing. I will modify the code 
so that it is consistent with the get_bound_dates function later

In [127]:
MIN_DEF, MAX_DEF = pd.Timedelta(days=0),pd.Timedelta(days=14)
          
# Get a number of orders that can be moved to other days
def get_cand_orders(df,date,col=False,num_orders=None):
    df_day = get_df_day(df,date)
    del_len = df_day["Completion Date"] - df_day["Start Date"]
    constr = (del_len >= MIN_DEF) & (del_len <= MAX_DEF) # can add more constraints if necessary
    if col == 'Start Date':
        constr = (del_len >= MIN_DEF) & (del_len <= MAX_DEF) & (df_day[col] == date) & (df_day['Shipper Region3'].str.contains('PQ'))
    elif col == 'End Date':
        constr = (del_len >= MIN_DEF) & (del_len <= MAX_DEF) & (df_day[col] == date) & (df_day['Consignee Region3'].str.contains('PQ'))
    res = df_day[constr]
    if num_orders != None:
        num_orders = int(min(len(res),num_orders)) # In cases where there are not as many jobs as requested
        return res.iloc[:num_orders,:]
    else: 
        return res

In [128]:
# Demo
df = DF_CLEAN
date1 = pd.Timestamp(2020,1,1,0)
date2 = pd.Timestamp(2020,1,7,0)

# get_cand_orders(df,date2) # Get all the candidate orders
get_cand_orders(df,date2,num_orders=5) # Get a certain number of the candidate orders on that day

,Priority,Requested Trailer Class,Shipper Region3,Consignee Region3,Start Date,Completion Date
14726,Standard,DRY,USMWMO,PQ2MON,2019-12-27,2020-01-07
4068,Standard,REEFER,USSWCS,PQ2MON,2019-12-31,2020-01-07
5490,Standard,HEATER,USSEE,PQ2QUE,2020-01-02,2020-01-07
13780,Expedited,DRY,ABSCAL,PQ2MON,2020-01-02,2020-01-07
14073,Standard,DRY,SK2SAS,PQ2S,2020-01-02,2020-01-07


In [129]:
# Actually reschedule an order in the scheduling table (by overwriting the date in column col)
# Returns the modified schedule and the updated counts
def move(df,index,to_date,col='Start Date'):
    df_resched = df.copy()
    df_resched.loc[index,col] = to_date
    df_recount = get_df_count(df_resched)
    return df_resched, df_recount

In [130]:
# Demo
df = DF_CLEAN
index = df.index[5]
to_date = df.loc[index,'Start Date'] + DAYS_1

df_resched,df_recount = move(df,index,to_date,'Start Date')
print(df.loc[index],'\n')
print(df_resched.loc[index])

Priority                              Standard
Requested Trailer Class                    DRY
Shipper Region3                         PQ2MON
Consignee Region3                       USMWIA
Start Date                 2020-04-08 00:00:00
Completion Date            2020-04-10 00:00:00
Name: 5, dtype: object 

Priority                              Standard
Requested Trailer Class                    DRY
Shipper Region3                         PQ2MON
Consignee Region3                       USMWIA
Start Date                 2020-04-09 00:00:00
Completion Date            2020-04-10 00:00:00
Name: 5, dtype: object


In [131]:
# Get the bound (earliest/latest) dates in between which an order have to start and complete.
# By default the assumption would be that the order must start in between 14 days prior to 
# originally scheduled all the way to 0 days before; and must completes within 1 day prior to 1 day after
# originally scheduled.
# Because of this, we based the completion date on the date in the ORIGINAL schedule (i.e DF_CLEAN)
def get_bound_dates(index):
    df = DF_CLEAN
    compl_date = df.loc[index,'Completion Date']
    e_start,l_start = compl_date - DAYS_14, compl_date - DAYS_0
    e_compl,l_compl = compl_date - DAYS_1, compl_date + DAYS_1
    return {"e_start":e_start,"l_start":l_start,"e_compl":e_compl,"l_compl":l_compl}

# Checks if an order violates any boundaries, especially after rescheduled
def check_bound_violations(df,index):
    start = df.loc[index,'Start Date'] 
    compl = df.loc[index,'Completion Date']
    e_start,l_start,e_compl,l_compl = get_bound_dates(index).values()
    return not (e_start <= start and start <= l_start and e_compl <= compl and compl <= l_compl)

In [132]:
# Demo
df = DF_CLEAN
index = df.index[5]

print(df.loc[index,'Start Date':'Completion Date'],'\n')
print(check_bound_violations(df,index))
get_bound_dates(index)

Start Date         2020-04-08 00:00:00
Completion Date    2020-04-10 00:00:00
Name: 5, dtype: object 

False


{'e_start': Timestamp('2020-03-27 00:00:00'),
 'l_start': Timestamp('2020-04-10 00:00:00'),
 'e_compl': Timestamp('2020-04-09 00:00:00'),
 'l_compl': Timestamp('2020-04-11 00:00:00')}

In [133]:
check_bound_violations(df,index)

False

In [134]:
# Get orders that have the same values in columns in col
# Used mostly to determine the shortest/longest expected time for order of the same type
def get_df_similar_orders(df,index,col=["Priority","Requested Trailer Class","Shipper Region3","Consignee Region3"]):
    order = df.loc[index,col]
    df_dup = df.loc[:,col]
    df_dup = df_dup[df_dup == order].dropna()
    return df.loc[df_dup.index,:]

In [135]:
# Demo
df = DF_CLEAN
index = df.index[5]

print(df.loc[index],'\n')
get_df_similar_orders(df,index)
# get_df_similar_orders(df,index,col=["Shipper Region3","Consignee Region3"]) # Only getting orders with same Shipper Region and Consignee Region

Priority                              Standard
Requested Trailer Class                    DRY
Shipper Region3                         PQ2MON
Consignee Region3                       USMWIA
Start Date                 2020-04-08 00:00:00
Completion Date            2020-04-10 00:00:00
Name: 5, dtype: object 



,Priority,Requested Trailer Class,Shipper Region3,Consignee Region3,Start Date,Completion Date
4,Standard,DRY,PQ2MON,USMWIA,2020-02-26,2020-03-02
5,Standard,DRY,PQ2MON,USMWIA,2020-04-08,2020-04-10
6,Standard,DRY,PQ2MON,USMWIA,2020-05-11,2020-05-13
7,Standard,DRY,PQ2MON,USMWIA,2020-12-01,2020-12-04
8,Standard,DRY,PQ2MON,USMWIA,2021-03-11,2021-03-14
9,Standard,DRY,PQ2MON,USMWIA,2021-03-17,2021-03-22
16075,Standard,DRY,PQ2MON,USMWIA,2019-12-03,2019-12-06
16076,Standard,DRY,PQ2MON,USMWIA,2019-12-18,2019-12-20
16077,Standard,DRY,PQ2MON,USMWIA,2019-12-29,2020-01-02
16078,Standard,DRY,PQ2MON,USMWIA,2020-01-13,2020-01-15


In [136]:
# Records the changes in days made to the "Start Date" and "Completion Date" of the original schedule
def get_df_sched_diff(df_sched,df_resched):
    df_st_diff = df_resched["Start Date"] - df_sched["Start Date"]
    df_cp_diff = df_resched["Completion Date"] - df_sched["Completion Date"]
    df = pd.concat([df_st_diff,df_cp_diff],axis=1)
    return df[df != DAYS_0].dropna(how='all').fillna(DAYS_0)

In [137]:
# Demo
date1 = pd.Timestamp(2021,1,1,0)

df = DF_CLEAN
index = df.index[5]
df_resched,df_recount = move(df_sched,index,date1,'Completion Date')

print(df.loc[index])
print(df_resched.loc[index])
get_df_sched_diff(df,df_resched)

Priority                              Standard
Requested Trailer Class                    DRY
Shipper Region3                         PQ2MON
Consignee Region3                       USMWIA
Start Date                 2020-04-08 00:00:00
Completion Date            2020-04-10 00:00:00
Name: 5, dtype: object
Priority                              Standard
Requested Trailer Class                    DRY
Shipper Region3                         PQ2MON
Consignee Region3                       USMWIA
Start Date                 2020-04-08 00:00:00
Completion Date            2021-01-01 00:00:00
Name: 5, dtype: object


,Start Date,Completion Date
5,0 days,266 days


# Algorithm

## Pseudo Code

Note:

    imb = inbound - outbound (how many spare trailers)  
    imb > 0 : there are spare trailers for the day (that can otherwise be utilized)    
    imb < 0 : there are shortage of trailers for the day (we likely have to hire outside trailers) 
    imb = 0 : there are just enough trailers 
    day.imb < next.imb : there are less spare trailers today than tomorrow/ or there are more of a shortage
    day.imb > next.imb : there are more spare trailers today than tomorrow/ or there are less of a shortage

Idea:
```
for date in all_days:
    next = day.next
    if (date.imb > 0): 
    # if we have spare trailers today
        if (next.imb > date.imb):
        # if tomorrow we have even more spare trailers than today
            #
            # We defer some outbound order today to tomorrow so we can utilize tomorrow's larger spare trailers        
            #
        elif (next.imb < date.imb):
        # if tomorrow there's less spare trailers, or even a shortage 
            #
            # We take in tomorrow outbound orders so we can utilize today's larger spare trailers
            #
            
    elif (date.imb < 0): 
    # if we are short of trailers today:
        if (next.imb > date.imb):
        # if we are less short of trailers tomorrow, or even have spare trailers
            #
            # We ask some inbound order to arrive sooner (today, as opposed to tomorrow)
            #
        elif (next.imb < date.imb):
        # if we are even more short of trailers tomorrow
            #
            # we allow some inbound order today to complete tomorrow
            #

```

## Code

In [138]:
def get_df_rescheduled(df,start_date=None,end_date=None):
    df_resched = get_df_span(df,start_date,end_date)
    df_recount = get_df_count(df,start_date,end_date,True)
    dates = df_recount.index
    
    for date in dates:
        next_ = date + DAYS_1
        date_imb = get_imb(df_recount,date)
        next_imb = get_imb(df_recount,next_)
        
        if date_imb > 0:
            if date_imb < next_imb: # if there's more truck coming in tomorrow
                num_orders = int((next_imb - date_imb)/2)
                fr,to = date,next_

                cand = get_cand_orders(df_resched,fr,'Start Date',num_orders)
                for index in cand.index:
                    e_start,l_start,e_compl,l_compl = get_bound_dates(index).values()
                    if e_start <= to and to <= l_start:
                        df_resched,df_recount = move(df_resched,index,to,col="Start Date")

            elif date_imb > next_imb:
                num_orders = int((date_imb - next_imb)/2)
                fr,to = next_,date

                cand = get_cand_orders(df_resched,fr,'Start Date',num_orders)
                for index in cand.index:
                    e_start,l_start,e_compl,l_compl = get_bound_dates(index).values()
                    if e_start <= to and to <= l_start:
                        df_resched,df_recount = move(df_resched,index,to,col="Start Date")

        elif date_imb < 0: 
            if date_imb < next_imb: # if there's more truck coming in tomorrow
                num_orders = int((next_imb - date_imb)/2)
                fr,to = next_,date
                
                cand = get_cand_orders(df_resched,fr,'Completion Date',num_orders)
                for index in cand.index:
                    e_start,l_start,e_compl,l_compl = get_bound_dates(index).values()
                    if e_compl <= to and to <= l_compl:
                        df_resched,df_recount = move(df_resched,index,to,col="Completion Date")

            elif date_imb > next_imb:
                num_orders = int((date_imb - next_imb)/2)
                fr,to = date,next_
                
                cand = get_cand_orders(df_resched,fr,'Completion Date',num_orders)
                for index in cand.index:
                    e_start,l_start,e_compl,l_compl = get_bound_dates(index).values()
                    if e_compl <= to and to <= l_compl:
                        df_resched,df_recount = move(df_resched,index,to,col="Completion Date")
    return df_resched

In [139]:
start_date = pd.Timestamp(2019,1,1,0)
end_date = pd.Timestamp(2019,3,31,0)

df = DF_CLEAN
df_resched = get_df_rescheduled(df,start_date,end_date)
df_resched

,Priority,Requested Trailer Class,Shipper Region3,Consignee Region3,Start Date,Completion Date
271,Standard,DRY,PQ2MON,ON2TOR,2019-03-29,2019-04-15
609,Standard,DRY,PQ2MON,USMEON,2019-03-26,2019-03-28
655,Standard,DRY,PQ2MON,USMEON,2019-03-25,2019-03-27
781,Standard,DRY,SK2SAS,PQ2MON,2019-03-20,2019-03-26
782,Standard,DRY,SK2SAS,PQ2MON,2019-03-22,2019-03-28
...,...,...,...,...,...,...
23591,Standard,REEFER,PQ2MON,ABSCAL,2019-03-29,2019-04-05
23605,Standard,REEFER,PQ2MON,ABSCAL,2019-03-20,2019-03-27
23624,Standard,DRY,PQ2MON,USILCH,2019-03-26,2019-03-27
23637,Standard,DRY,PQ2MON,USMWMO,2019-03-25,2019-03-28


In [ ]:
df_sched = get_df_span()